In [1]:
import os
import sys
import subprocess
from google.colab import drive

# 1. Mount Google Drive
print("🔌 Mounting Google Drive...")
drive.mount('/content/drive')

# 2. Install Training Libraries
# We need 'peft' for LoRA and 'datasets' for MagicBrush
print("⏳ Installing libraries (this takes ~45s)...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                       "git+https://github.com/huggingface/diffusers.git",
                       "accelerate", "transformers", "datasets", "peft", "bitsandbytes"])
print("✅ Environment Ready.")

🔌 Mounting Google Drive...
Mounted at /content/drive
⏳ Installing libraries (this takes ~45s)...
✅ Environment Ready.


In [2]:
import torch
import os
import shutil
import gc
import datasets
from accelerate import Accelerator
from diffusers import DDPMScheduler, UNet2DConditionModel, AutoencoderKL
from peft import LoraConfig, get_peft_model, set_peft_model_state_dict
from transformers import CLIPTokenizer, CLIPTextModel
from datasets import load_dataset
from torchvision import transforms
from tqdm.auto import tqdm
from safetensors.torch import load_file

# ================= CONFIGURATION =================
# We save directly to Drive so we never lose progress
CHECKPOINT_DIR = "/content/drive/MyDrive/Projects/Image-Editing-by-Natural-Language-Constraints/checkpoints"
FINAL_OUTPUT_DIR = "/content/drive/MyDrive/Projects/Image-Editing-by-Natural-Language-Constraints/lora_instruction_tuned"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

MAX_STEPS = 2000
SAVE_INTERVAL = 10
CLEANUP_INTERVAL = 30
# =================================================

def cleanup_disk():
    """Frees up disk space by clearing caches."""
    print("🧹 Cleaning up disk space...")
    try:
        hf_cache = "/root/.cache/huggingface/datasets"
        if os.path.exists(hf_cache):
            shutil.rmtree(hf_cache)
            os.makedirs(hf_cache, exist_ok=True)
        gc.collect()
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"   ⚠️ Cleanup warning: {e}")

print("🚀 Starting Robust LoRA Training (500 Steps)...")

# 1. Setup Accelerator
accelerator = Accelerator(gradient_accumulation_steps=4, mixed_precision="fp16")
device = accelerator.device
MODEL_ID = "runwayml/stable-diffusion-v1-5"

# 2. Load MagicBrush
print("📥 Loading MagicBrush...")
dataset = load_dataset("osunlp/MagicBrush", split="train[:800]")

# 3. Load Models
print("📦 Loading Stable Diffusion...")
tokenizer = CLIPTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet")
scheduler = DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")

vae.requires_grad_(False)
text_encoder.requires_grad_(False)
unet.requires_grad_(False)

# 4. Inject LoRA
print("💉 Injecting LoRA Adapters...")
lora_config = LoraConfig(
    r=16, # Higher rank for better capacity
    lora_alpha=32,
    target_modules=["to_k", "to_q", "to_v", "to_out.0"],
    init_lora_weights="gaussian"
)
unet = get_peft_model(unet, lora_config)

# 5. Resume Logic
start_step = 0
checkpoints = sorted([int(x.split("_")[1]) for x in os.listdir(CHECKPOINT_DIR) if x.startswith("step_")])
if checkpoints:
    latest_step = checkpoints[-1]
    ckpt_path = os.path.join(CHECKPOINT_DIR, f"step_{latest_step}")
    print(f"🔄 Resuming from checkpoint: {ckpt_path}")
    state_dict = load_file(os.path.join(ckpt_path, "adapter_model.safetensors"))
    set_peft_model_state_dict(unet, state_dict)
    start_step = latest_step
    print(f"   ✅ Resumed at Step {start_step}")
else:
    print("✨ Starting fresh training.")

unet.print_trainable_parameters()

# 6. Optimizer
optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-4)
unet, optimizer, text_encoder, vae = accelerator.prepare(
    unet, optimizer, text_encoder, vae
)

# 7. Training Loop
unet.train()
transform = transforms.Compose([
    transforms.Resize((512,512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

print(f"🔥 Training from Step {start_step} to {MAX_STEPS}...")
progress_bar = tqdm(range(start_step, MAX_STEPS))
data_iter = iter(dataset)

for global_step in range(start_step, MAX_STEPS):
    try:
        batch = next(data_iter)
    except StopIteration:
        data_iter = iter(dataset)
        batch = next(data_iter)

    with accelerator.accumulate(unet):
        target_pixels = transform(batch['target_img'].convert("RGB")).unsqueeze(0).to(device)
        latents = vae.encode(target_pixels).latent_dist.sample() * 0.18215

        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (1,), device=device).long()
        noisy_latents = scheduler.add_noise(latents, noise, timesteps)

        tokens = tokenizer(batch['instruction'], padding="max_length", truncation=True, return_tensors="pt").input_ids.to(device)
        encoder_hidden_states = text_encoder(tokens)[0]

        model_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample
        loss = torch.nn.functional.mse_loss(model_pred, noise, reduction="mean")
        accelerator.backward(loss)
        optimizer.step()
        optimizer.zero_grad()

    progress_bar.update(1)
    progress_bar.set_description(f"Loss: {loss.item():.4f}")

    if (global_step + 1) % SAVE_INTERVAL == 0:
        save_path = os.path.join(CHECKPOINT_DIR, f"step_{global_step + 1}")
        print(f"\n💾 Saving checkpoint to: {save_path}")
        unet.save_pretrained(save_path)

    if (global_step + 1) % CLEANUP_INTERVAL == 0:
        cleanup_disk()

print(f"\n🎉 Training Complete! Saving final model to: {FINAL_OUTPUT_DIR}")
unet.save_pretrained(FINAL_OUTPUT_DIR)
print("✅ Done.")

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


🚀 Starting Robust LoRA Training (500 Steps)...
📥 Loading MagicBrush...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/51 [00:00<?, ?it/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

💉 Injecting LoRA Adapters...
🔄 Resuming from checkpoint: /content/drive/MyDrive/Projects/Image-Editing-by-Natural-Language-Constraints/checkpoints/step_540
   ✅ Resumed at Step 540
trainable params: 3,188,736 || all params: 862,709,700 || trainable%: 0.3696
🔥 Training from Step 540 to 2000...


  0%|          | 0/1460 [00:00<?, ?it/s]


💾 Saving checkpoint to: /content/drive/MyDrive/Projects/Image-Editing-by-Natural-Language-Constraints/checkpoints/step_550

💾 Saving checkpoint to: /content/drive/MyDrive/Projects/Image-Editing-by-Natural-Language-Constraints/checkpoints/step_560

💾 Saving checkpoint to: /content/drive/MyDrive/Projects/Image-Editing-by-Natural-Language-Constraints/checkpoints/step_570
🧹 Cleaning up disk space...

💾 Saving checkpoint to: /content/drive/MyDrive/Projects/Image-Editing-by-Natural-Language-Constraints/checkpoints/step_580

💾 Saving checkpoint to: /content/drive/MyDrive/Projects/Image-Editing-by-Natural-Language-Constraints/checkpoints/step_590

💾 Saving checkpoint to: /content/drive/MyDrive/Projects/Image-Editing-by-Natural-Language-Constraints/checkpoints/step_600
🧹 Cleaning up disk space...

💾 Saving checkpoint to: /content/drive/MyDrive/Projects/Image-Editing-by-Natural-Language-Constraints/checkpoints/step_610

💾 Saving checkpoint to: /content/drive/MyDrive/Projects/Image-Editing-by-Na